In [200]:
import os
import sys
import glob
import boto3
import re

In [2]:
!pwd

/Users/home/Documents/dend/Data-Engineering-ND/Capstone/notebook


In [18]:
path = r'/Users/home/Documents/dend/Data-Engineering-ND/Capstone/data'
for files in os.listdir(path):
    print(files)

.DS_Store
18-83510-I94-Data-2016
airport-codes.csv
us-cities-demographics.csv


In [9]:
for f in glob.glob(path+"/*"):
    print(f)

/Users/home/Documents/dend/Data-Engineering-ND/Capstone/data/18-83510-I94-Data-2016
/Users/home/Documents/dend/Data-Engineering-ND/Capstone/data/airport-codes.csv
/Users/home/Documents/dend/Data-Engineering-ND/Capstone/data/us-cities-demographics.csv


In [19]:
for _, _, f in os.walk(path):
    print(f)

['.DS_Store', 'airport-codes.csv', 'us-cities-demographics.csv']
['i94_nov16_sub.sas7bdat', 'i94_dec16_sub.sas7bdat', 'i94_sep16_sub.sas7bdat', 'i94_aug16_sub.sas7bdat', 'i94_may16_sub.sas7bdat', 'i94_jun16_sub.sas7bdat', 'i94_oct16_sub.sas7bdat', 'i94_apr16_sub.sas7bdat', 'i94_jan16_sub.sas7bdat', 'i94_jul16_sub.sas7bdat', 'i94_mar16_sub.sas7bdat', 'i94_feb16_sub.sas7bdat']


In [17]:
path

'/Users/home/Documents/dend/Data-Engineering-ND/Capstone/data/18-83510-I94-Data-2016'

In [80]:
def copy_files(
            source_path,
            file_ext,
            s3_bucket,
            s3_sas_key,
            src_files=None):
    for path,_,files in os.walk(source_path):
        try:
            for file in files:
                if src_files is not None:
                    if file.endswith(file_ext) and file in src_files:
                        print(file)
                    else:
                        print(f'File {file} doesnt end with extension {file_ext}')
                else:
                    if file.endswith(file_ext):
                        print(f'Copied file {file} into S3 bucket...')
                    else:
                        print(f'File {file} doesnt end with extension {file_ext}')

        except Exception as e:
            print(f'Failed to copy {file} into S3 bucket...')

In [82]:
path = r'/Users/home/Documents/dend/Data-Engineering-ND/Capstone/data'
src_files = ["i94_apr16_sub.sas7bdat", "i94_aug16_sub.sas7bdat"]
copy_files(path, 'sas7bdat', 'something', 'anotherthing', src_files)

File .DS_Store doesnt end with extension sas7bdat
File airport-codes.csv doesnt end with extension sas7bdat
File us-cities-demographics.csv doesnt end with extension sas7bdat
File i94_nov16_sub.sas7bdat doesnt end with extension sas7bdat
File i94_dec16_sub.sas7bdat doesnt end with extension sas7bdat
File i94_sep16_sub.sas7bdat doesnt end with extension sas7bdat
i94_aug16_sub.sas7bdat
File i94_may16_sub.sas7bdat doesnt end with extension sas7bdat
File i94_jun16_sub.sas7bdat doesnt end with extension sas7bdat
File i94_oct16_sub.sas7bdat doesnt end with extension sas7bdat
i94_apr16_sub.sas7bdat
File i94_jan16_sub.sas7bdat doesnt end with extension sas7bdat
File i94_jul16_sub.sas7bdat doesnt end with extension sas7bdat
File i94_mar16_sub.sas7bdat doesnt end with extension sas7bdat
File i94_feb16_sub.sas7bdat doesnt end with extension sas7bdat


In [49]:
'i94_nov16_sub.sas7bdat'.endswith('sas7bdat')

True

In [1]:
import boto3
import configparser
import os
import time
import json

In [2]:
def create_client(service, region, access_key_id, secret_access_key):
    return boto3.client(service, region_name=region, 
                aws_access_key_id=access_key_id,
                aws_secret_access_key=secret_access_key)

In [182]:
s3_client = create_client('s3',
                         region='us-west-2',
                         access_key_id='your access key',
                         secret_access_key='your secret access key')

In [87]:
def create_s3bucket(bucket_name, s3_client):
    s3_client.create_bucket(Bucket=bucket, CreateBucketConfiguration='us-west-2')

In [215]:
def upload_files(s3_client, source_path, bucket_name, key, file_ext='', src_files=None):
    pattern = source_path.split('/')[-1]
    for path, _, files in os.walk(source_path):
        print(path, files)
        match=re.search(pattern, path)
        try:
            for _file in files:
                # Added this condition to limit
                # the SAS files to only 2 files
                if src_files is not None:
                    if _file in src_files:
                        s3_client.upload_file(Filename=os.path.join(path, _file), Bucket=bucket_name,  Key=os.path.join(key, _file))
                        print(f'Copied file {_file} into S3 bucket...')
                    else:
                        print(f'File {_file} is not in the list specified...')
                else:
                    if _file.endswith(file_ext):
                        s3_client.upload_file(Filename=os.path.join(path, _file), Bucket=bucket_name, 
                                              Key=os.path.join(key, _file))
                        print(f'Copied file {_file} into S3 bucket...')
                    else:
                        # print(f'File {_file} doesnt end with extension {file_ext}')
                        if _file == '.DS_Store': continue
                        s3_client.upload_file(os.path.join(path, _file), Bucket=bucket_name,
                                              Key=os.path.join(key, path[match.end() + 1:], _file))
                        print(f'Copied file {_file} into S3 bucket...')

        except Exception as e:
            print(f'Failed to copy {_file} into S3 bucket...')
            print(e)
            

In [164]:
parser = configparser.ConfigParser()
# cfg_rel_path = os.path.dirname(os.path.realpath(__file__))
# print(cfg_rel_path)
# f = os.path.relpath('/Users/home/Documents/dend/Data-Engineering-ND/Capstone/config/etl_config.cfg', cfg_rel_path)
# print(f)
cfg = r'/Users/home/Documents/dend/Data-Engineering-ND/Capstone/config/etl_config.cfg'
try:
    parser.read(cfg)
except Exception as e:
    print('Cannot find etl_config.cfg file...')
    self.log.error('Cannot find etl_config.cfg file...')
    raise

aws_region = parser['AWS']['aws_region']
base_dir = parser['LOCAL']['base_dir']
data_dir = parser['LOCAL']['data_dir']
dict_dir = parser['LOCAL']['dict_dir']
output_dir = parser['LOCAL']['output_dir']
log_dir = parser['LOCAL']['log_dir']

# base_dir = parser['DOCKER']['base_dir']
# data_dir = parser['DOCKER']['data_dir']
# dict_dir = parser['DOCKER']['dict_dir']

data_dir = os.path.join(base_dir, data_dir)
dict_dir = os.path.join(base_dir, dict_dir)
log_dir = os.path.join(base_dir, log_dir)
output_dir = os.path.join(base_dir, output_dir)
# files = json.loads(parser['LOCAL']['input_files'])
files = json.loads(parser['DOCKER']['input_files'])

s3_bucket = parser['S3']['s3_bucket']
s3_sas_key = parser['S3']['s3_sas_key']
s3_csv_key = parser['S3']['s3_csv_key']
s3_dict_key = parser['S3']['s3_dict_key']
s3_output_key = parser['S3']['s3_output_key']
s3_log_key = parser['S3']['S3_log_key']

In [108]:
# path = r'/Users/home/Documents/dend/Data-Engineering-ND/Capstone/data'
# src_files = ["i94_apr16_sub.sas7bdat", "i94_aug16_sub.sas7bdat"]
# upload_files(
#     s3_client,
#     dict_dir,
#     'SAS',
#     s3_bucket,
#     s3_dict_key
# )

In [109]:
# upload_files(
#     s3_client,
#     log_dir,
#     s3_bucket,
#     s3_log_key,
#     file_ext='log'
# )

In [110]:
upload_files(
    s3_client=s3_client,
    source_path=os.path.join(output_dir, 'i94_visa'),
    file_ext='ignore',
    bucket_name=s3_bucket,
    key=s3_output_key,
)

/Users/home/Documents/dend/Data-Engineering-ND/Capstone/output/i94_visa ['.part-00005-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv.crc', 'part-00000-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv', '._SUCCESS.crc', '.part-00010-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv.crc', 'part-00015-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv', 'part-00005-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv', '.part-00015-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv.crc', '_SUCCESS', 'part-00010-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv', '.part-00000-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv.crc']
Copied file .part-00005-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv.crc into S3 bucket...
Copied file part-00000-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv into S3 bucket...
Copied file ._SUCCESS.crc into S3 bucket...
Copied file .part-00010-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv.crc into S3 bucket...
Copied file part-00015-aa20cb3d-5020-4a74-971c-96504130b654-c000.csv into S3 bucket

In [90]:
os.path.join(output_dir, 'i94_visa')

'/Users/home/Documents/dend/Data-Engineering-ND/Capstone/output/i94_visa'

In [170]:
s3 = boto3.resource('s3', aws_access_key_id='your access key',
         aws_secret_access_key= 'your secret access key',
                   region_name=aws_region)

try:
    s3.meta.client.head_bucket(Bucket=s3_bucket)
    print(f"S3 bucket {s3_bucket_name} exists...")
except Exception as e:
    print(f"Bucket {s3_bucket} doesn't exist...")

Bucket immigrations-analytics doesn't exist...


In [168]:
bucket = s3.Bucket(s3_bucket)

In [169]:
bucket

s3.Bucket(name='immigrations-analytics')

In [171]:
try:
    r = s3_client.get_object(Bucket=s3_bucket, Key='config/etl_config.cfg')
except Exception as e:
    print(e)

In [172]:
r

{'ResponseMetadata': {'RequestId': 'CZWRNZM4R920HABG',
  'HostId': 'srYrvgdmC2ERucchURaM6TDOFCHEt9wJYtSU3tc0+mr0x360VrCN/k3LyrYR5MW9l4swQT1bZ8Q=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'srYrvgdmC2ERucchURaM6TDOFCHEt9wJYtSU3tc0+mr0x360VrCN/k3LyrYR5MW9l4swQT1bZ8Q=',
   'x-amz-request-id': 'CZWRNZM4R920HABG',
   'date': 'Sun, 09 May 2021 08:15:22 GMT',
   'last-modified': 'Sun, 09 May 2021 08:00:28 GMT',
   'etag': '"c18253d8fee209e0f14c55caff2f014f"',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'content-length': '1624',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 5, 9, 8, 0, 28, tzinfo=tzutc()),
 'ContentLength': 1624,
 'ETag': '"c18253d8fee209e0f14c55caff2f014f"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7face3607f60>}

In [186]:
# as s3.meta.client.head_bucket API is working incorrectly
def create_s3_bucket(s3_client, bucket_name, region='us-west-2'):
    """
    Create an s3 bucket
    :params s3_client: An S3 client object
    :params bucket_name: A unique bucket
    :params region: A valid AWS region
    Returns - None
    """
    try:
        s3_client.create_bucket(Bucket=bucket_name,
                                CreateBucketConfiguration={'LocationConstraint': region})
    except Exception as e:
        print('Bucket creation has failed...')
        print(e)
        raise

In [187]:
create_s3_bucket(s3_client, s3_bucket)

Bucket creation has failed...
An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [190]:
c = boto3.session.Session()
s3_resource = c.resource('s3')
s3_resource.meta.client.head_bucket(Bucket=s3_bucket)

ClientError: An error occurred (403) when calling the HeadBucket operation: Forbidden

In [194]:
s3.Bucket('sds').creation_date is None

True

In [203]:
pattern = output_dir.split('/')[-1]
for path, _, files in os.walk(output_dir):
    print(path, files)

/Users/home/Documents/dend/Data-Engineering-ND/Capstone/output []
/Users/home/Documents/dend/Data-Engineering-ND/Capstone/output/i94_visa ['.part-00005-7344b2f8-15a0-487e-a448-6897e7df747c-c000.csv.crc', 'part-00005-7344b2f8-15a0-487e-a448-6897e7df747c-c000.csv', '._SUCCESS.crc', '.part-00010-7344b2f8-15a0-487e-a448-6897e7df747c-c000.csv.crc', 'part-00010-7344b2f8-15a0-487e-a448-6897e7df747c-c000.csv', 'part-00000-7344b2f8-15a0-487e-a448-6897e7df747c-c000.csv', '.part-00015-7344b2f8-15a0-487e-a448-6897e7df747c-c000.csv.crc', 'part-00015-7344b2f8-15a0-487e-a448-6897e7df747c-c000.csv', '_SUCCESS', '.part-00000-7344b2f8-15a0-487e-a448-6897e7df747c-c000.csv.crc']
/Users/home/Documents/dend/Data-Engineering-ND/Capstone/output/i94_flights ['.part-00187-324f9b01-a475-4025-85aa-c880d70039e0-c000.csv.crc', 'part-00198-324f9b01-a475-4025-85aa-c880d70039e0-c000.csv', 'part-00136-324f9b01-a475-4025-85aa-c880d70039e0-c000.csv', 'part-00014-324f9b01-a475-4025-85aa-c880d70039e0-c000.csv', '.part-0015

In [197]:
output_dir

'/Users/home/Documents/dend/Data-Engineering-ND/Capstone/output'

In [209]:
s = '/Users/home/Documents/dend/Data-Engineering-ND/Capstone/output/i94_dates/i94_year=2016/i94_month=4'
g = re.search(pattern, s)

In [212]:
g.start()

56

In [214]:
s[g.end() + 1:]

'i94_dates/i94_year=2016/i94_month=4'